In [1]:
''' Predict vegitables code here '''
#imports
import keras
from keras.models import Model
from keras import applications, optimizers
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.layers import Dense, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
import os 

Using TensorFlow backend.


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/PicToRecipe"

/content/drive/My Drive/Colab Notebooks/PicToRecipe


In [0]:
class PredictRawVeggies:

    ###########################################################################
    def __init__(self):
        #Load the model
        self.img_width = 224
        self.img_height = 224

        self.create_model()

        self.model_final._make_predict_function()
        #get the labels
        df_labels = pd.read_csv("labels.csv")
        self.labels= list(df_labels['Label'])

    ############################################################################
    def create_model(self):
        #load the model
        model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (self.img_width, self.img_height, 3))
        #disable few layers
        for layer in model.layers[:5]:
            layer.trainable = False

        #Add layers
        x = model.output
        x = Flatten()(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.5)(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation="relu")(x)
#         x = Dropout(0.2)(x)
#         x = Dense(128, activation="relu")(x)
        #Add output layer
        predictions = Dense(26, activation="softmax")(x)
        #create the final model
        self.model_final = Model(inputs = model.input, outputs = predictions)
        #load the weights
        self.model_final.load_weights("vgg19_4_50.h5")
        # compile the model
        self.model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

    ######################################################################
    def call_predict(self, images, folder):

        predicted = []
        labels_df = pd.read_csv('labels.csv',index_col=False)

        #predict
        for image_name in images:
            image_path = folder+ "/" + image_name
#             print(f"imagepath: {image_path}")
            test_image = keras.preprocessing.image.load_img(image_path, target_size=(224,224), grayscale=False)
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis=0)
            test_image = preprocess_input(test_image)
#             predictions.append([image_name,self.model_final.predict(test_image)])
#             # print(test_image)
            predict = self.model_final.predict(test_image)
#             for itr in range(len(predict[0])):
            predicted.append([image_name,labels_df[labels_df.Index==np.argmax(predict[0])].Label])
#               predicted.append(labels_df[labels_df.Index==np.argmax(predict[0][itr])].Label )
#             zip_pred= zip(predict[0], self.labels)
#             print(zip_pred)
#             for pred_value, pred in zip_pred:
#                 if (pred_value > 0.7):
#                   predictions.append((image_name, pred))

        return predicted

In [0]:
my_pred = PredictRawVeggies()

In [0]:
images_list = os.listdir("./DataSet/single_prediction")
output = my_pred.call_predict(images_list,"./DataSet/single_prediction")


In [9]:
for itr in range(len(output)):
  print('for image '+ str(output[itr][0]) + ' predicted ' + str(output[itr][1]))

for image corn.jpg predicted 11    Corn
Name: Label, dtype: object
for image Green-Tomatoes-1920x850.jpg predicted 22    Green Tomato
Name: Label, dtype: object
for image green_beans.jpg predicted 6    Green Beans
Name: Label, dtype: object
for image testbrocoli.jpg predicted 15    Broccoli
Name: Label, dtype: object
for image mushrooms_sliced.jpg predicted 10    Mushroom
Name: Label, dtype: object
for image Tomato_3.jpg predicted 3    Red Onion
Name: Label, dtype: object
for image caulif.jpg predicted 9    Brussels Sprouts
Name: Label, dtype: object
for image sweet-potatoes.jpg predicted 3    Red Onion
Name: Label, dtype: object
for image green_pepper.jpg predicted 24    Green pepper
Name: Label, dtype: object
for image tomato_vine.jpg predicted 3    Red Onion
Name: Label, dtype: object
for image spring_onion.jpg predicted 18    Spring Onion
Name: Label, dtype: object
for image red_pepper.jpg predicted 2    Red pepper
Name: Label, dtype: object
for image peas.jpg predicted 5    Peas
N